In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [25]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [15]:
train_transation = pd.read_csv("./data/train_transaction.csv")

In [37]:
filtered_transations = train_transation.iloc[:,0:17]

In [38]:
print(filtered_transations.shape)
print(train_transation.shape)

(590540, 17)
(590540, 394)


In [40]:
filtered_transations.head(20)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain
0,2987000,0,86400,68.500,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN
1,2987001,0,86401,29.000,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN
2,2987002,0,86469,59.000,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN
3,2987003,0,86499,50.000,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN
4,2987004,0,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN
5,2987005,0,86510,49.000,W,5937,555.0,150.0,visa,226.0,debit,272.0,87.0,36.0,NaN,gmail.com,NaN
6,2987006,0,86522,159.000,W,12308,360.0,150.0,visa,166.0,debit,126.0,87.0,0.0,NaN,yahoo.com,NaN
7,2987007,0,86529,422.500,W,12695,490.0,150.0,visa,226.0,debit,325.0,87.0,NaN,NaN,mail.com,NaN
8,2987008,0,86535,15.000,H,2803,100.0,150.0,visa,226.0,debit,337.0,87.0,NaN,NaN,anonymous.com,NaN
9,2987009,0,86536,117.000,W,17399,111.0,150.0,mastercard,224.0,debit,204.0,87.0,19.0,NaN,yahoo.com,NaN


## Conversion to Categorical Values
Convert `ProductCD, card4, card6,P_emaildomain, R_emaildomain` to categorical.

In [55]:
labelencoder = LabelEncoder()
filtered_transations['ProductCD'] = labelencoder.fit_transform(filtered_transations['ProductCD'])
filtered_transations['card4'] = labelencoder.fit_transform(filtered_transations['card4'].astype(str))
filtered_transations['card6'] = labelencoder.fit_transform(filtered_transations['card6'].astype(str))
filtered_transations['P_emaildomain'] = labelencoder.fit_transform(filtered_transations['P_emaildomain'].astype(str))
filtered_transations['R_emaildomain'] = labelencoder.fit_transform(filtered_transations['R_emaildomain'].astype(str))

In [56]:
filtered_transations[filtered_transations['isFraud']==1].head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain
203,2987203,1,89760,445.000,4,18268,583.0,150.0,4,226.0,1,251.0,87.0,NaN,NaN,2,32
240,2987240,1,90193,37.098,0,13413,103.0,185.0,4,137.0,1,NaN,NaN,NaN,NaN,19,19
243,2987243,1,90246,37.098,0,13413,103.0,185.0,4,137.0,1,NaN,NaN,NaN,NaN,19,19
245,2987245,1,90295,37.098,0,13413,103.0,185.0,4,137.0,1,NaN,NaN,NaN,NaN,19,19
288,2987288,1,90986,155.521,0,16578,545.0,185.0,4,226.0,1,NaN,NaN,NaN,NaN,36,36


In [57]:
onehotencoder = OneHotEncoder(categorical_features = [5])

In [61]:
filtered_transations.fillna(0, inplace=True)

In [62]:
X = onehotencoder.fit_transform(filtered_transations).toarray()

/Users/carl.sharpe/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/carl.sharpe/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [63]:
type(X)

numpy.ndarray

In [66]:
X[0:20]

array([[ 0.,  0.,  0., ...,  0., 32., 32.],
       [ 0.,  0.,  0., ...,  0., 16., 32.],
       [ 0.,  0.,  0., ...,  0., 36., 32.],
       ...,
       [ 0.,  0.,  0., ...,  0., 54., 32.],
       [ 0.,  0.,  0., ...,  0., 16., 32.],
       [ 0.,  0.,  0., ...,  0., 16., 32.]])